In [3]:
from tqdm.notebook import tqdm

import pandas as pd
import numpy as np
import pytorch_lightning as pl
from scipy.spatial.distance import cosine

from evaluate import load_pitchclass2vec_model

from collections import defaultdict
from itertools import groupby
import re

import biab

RANDOM_SEED = 42
pl.seed_everything(seed=RANDOM_SEED)

Global seed set to 42


42

# Load data

In [49]:
def extract_id(s):
    x = re.findall("-(\d+)\w?\.", str(s))
    return int(x[0]) if len(x) > 0 else None

df = pd.read_csv("./realbook/RealBook/songlist.csv", names=["title", "path"], sep=";")
df["id"] = df.path.apply(extract_id)

In [59]:
id2chords = defaultdict(list)
id2title = defaultdict(str)
corpus = list()
for idx, (song, path, id) in df.iterrows():
    try:
        chords = biab.biab_chords(f"./realbook/RealBook/{path.lower()}")
        chords = [x[-1] for x in chords]
        
        if len(chords) > 0:
            id2chords[id].append(chords)
            id2title[id] = song
            corpus.append((id, chords))
    except:
        pass

Exception: Inconsistent number of chord types and roots


In [61]:
titles, chords = zip(*corpus)
titles = np.array(titles)
chords = np.array(chords)

/tmp/ipykernel_1201613/3557118382.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  chords = np.array(chords)


# Cover song detection

State of the art is TPSD with 

first_tier 0.74 e second tier 0.77.

In [62]:
def embed_chord(p2v, c):
    try:
        return p2v[c]
    except:
        return p2v["N"]

In [63]:
#p2v = load_pitchclass2vec_model("rdf", "randomwalk-rdf2vec", "out/rdf2vec_best/model.ckpt")
#p2v = load_pitchclass2vec_model("timed-root-interval", "emb-weighted-fasttext", "out/rootinterval_best/model.ckpt")
p2v = load_pitchclass2vec_model("text", "fasttext", "out/fasttext_best/model.ckpt")

In [64]:
corpus_embedded = np.stack([
    np.mean(np.array([embed_chord(p2v, c) for c in x]), axis=0) for x in chords
])

In [65]:
def get_top_k(title, k = 5):
    sim = np.apply_along_axis(cosine, 1, corpus_embedded, a)
    top_idxs = sim.argsort()[:k]
    return top_idxs, sim[top_idxs]

In [67]:
first_tiers = list()
second_tiers = list()

for t, _ in filter(lambda x: len(x[1]) > 1, id2chords.items()):
    C_idxs = np.where(titles == t)[0]
    first_tier = list()
    second_tier = list()
    
    for C_idx in C_idxs:
        query = corpus_embedded[C_idx]        
        sim = np.apply_along_axis(cosine, 1, corpus_embedded, query).argsort()
        
        # compute first tier
        top_idxs = sim[:len(C_idxs) - 1]
        first_tier.append(len(set(top_idxs).intersection(C_idxs)) / (len(C_idxs) - 1))
        
        # compute second tier
        top_idxs = sim[:(2 * len(C_idxs)) - 1]
        second_tier.append(len(set(top_idxs).intersection(C_idxs)) / (len(C_idxs) - 1))
    
    first_tiers.append(np.mean(first_tier))
    second_tiers.append(np.mean(second_tier))

print("First tier", np.mean(first_tiers))
print("Second tier", np.mean(second_tiers))

First tier 0.8242971887550201
Second tier 1.1110441767068273


In [69]:
len(list(filter(lambda x: len(x[1]) > 1, id2chords.items())))

83